In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd


300

# Scraping the data from Metacritic - loading 10 pages of 100 movies each

In [ ]:
meta_data = []
meta_url="https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page="
for i in range(10):
    meta_response = requests.get(f'{meta_url}{i}')
    meta_soup = BeautifulSoup(meta_response.content, 'lxml')   
    meta_data += meta_soup.find_all("td",attrs={"class":"clamp-summary-wrap"})
len(meta_data)

# Setting up relevant values into arrays , using scraping 

In [2]:
rank = []
title = []
metascore = []
userscore = []
year = []
for movie in meta_data:
    rank.append(movie.find("span",attrs={"class":"title numbered"}).text.strip().replace(".",""))
    title.append(movie.find("a",attrs={"class":"title"}).text.strip())
    year.append(movie.find("div",attrs={"class":"clamp-details"}).text.split()[2])
    for m in movie.find_all("div",attrs={"class":"browse-score-clamp"}):
        metascore.append(m.text.split()[1])
        userscore.append(m.text.split()[4])
        


# Setting up a Dataframe with the info we collected 

In [3]:
df = pd.DataFrame(title,columns=['Title'])
df['Metascore']=metascore
df['Meta User Score']=userscore
df['Metacritic Rank']=rank
df['Year']=year
df

,Title,Metascore,Meta User Score,Metacritic Rank,Year
0,Shoah: Four Sisters,93,tbd,101,2018
1,Duck Soup,93,8.8,102,1933
2,Days of Heaven,93,8.6,103,1978
3,Toni Erdmann,93,7.1,104,2016
4,Yi Yi,93,8.4,105,2000
...,...,...,...,...,...
295,All Is Lost,87,6.6,396,2013
296,Hereditary,87,7.2,397,2018
297,Listen to Me Marlon,87,8.4,398,2015
298,Gideon's Army,87,8.5,399,2013


# Data Cleaning & Convertions

In [4]:
df['Metascore'] = df['Metascore'].astype("float64")/10
df['Meta User Score'] = pd.to_numeric(df['Meta User Score'], errors='coerce')
df['Metacritic Rank'] = df['Metacritic Rank'].astype("int64")
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df.dropna(inplace=True)
df['Year'] = df['Year'].astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 1 to 299
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            265 non-null    object 
 1   Metascore        265 non-null    float64
 2   Meta User Score  265 non-null    float64
 3   Metacritic Rank  265 non-null    int64  
 4   Year             265 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 12.4+ KB


In [5]:
df

,Title,Metascore,Meta User Score,Metacritic Rank,Year
1,Duck Soup,9.3,8.8,102,1933
2,Days of Heaven,9.3,8.6,103,1978
3,Toni Erdmann,9.3,7.1,104,2016
4,Yi Yi,9.3,8.4,105,2000
6,The Best Years of Our Lives,9.3,8.4,107,1946
...,...,...,...,...,...
295,All Is Lost,8.7,6.6,396,2013
296,Hereditary,8.7,7.2,397,2018
297,Listen to Me Marlon,8.7,8.4,398,2015
298,Gideon's Army,8.7,8.5,399,2013


# Saving the dataframe we made into a csv file to load
### we saved it once becouse the website keep giving bans for scaping.

In [7]:
#df.to_csv('metacriticData.csv')